In [2]:
import cv2
import numpy as np
from skimage.measure import label, regionprops
import os
# import parameters as params
import multiprocessing
import bisect
import time
import random

# from components import *
from skimage import measure
import matplotlib.pyplot as plt

from fourier import *

In [3]:
def connected_components(img):
    lbl = label(img)
    blobs = regionprops(lbl)
    return blobs
def my_preprocessing(img):
    img_denoised = denoised(img)
    img_bin, th = binarized(img_denoised)
    return img_bin

def read_input(path):
    img = cv2.imread(path)
    img = img[:,:,0]
    print('img shape: ',img.shape)
    return img

def denoised(img):
    denoised_img = cv2.fastNlMeansDenoising(img,None,30,7,30)
#     cv2.imwrite(os.path.join(output_path, 'denoised.jpg'),denoised_img)
    return denoised_img

def binarized(img, threshold = -1):
    th, img_th = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    print(' OT threshold: ',th)
    #print('threshold using: ',th+10)
    th, img_th = cv2.threshold(img, th+10, 255 , cv2.THRESH_BINARY)
    #cv2.imwrite(os.path.join(output_path,'binarized.jpg'),img_th)
    #print('threshold using: ',th)
    return img_th, th

In [4]:
class shape_packet:

    def __init__(self):
        self.horizontal_threshold = 20
        self.packet_type = 0
        self.bbox = (0,0,0,0)
        self.blob_img = []
        self.centroid = []
        
    def set_centroid(self):
        total_rows, total_cols = self.blob_img.shape

        # Calculate the sum of x and y coordinates
        sum_x = 0
        sum_y = 0
        total_points = 0
    
        for row in range(total_rows):
            for col in range(total_cols):
                if self.blob_img[row][col]:
                    sum_y += col
                    sum_x += row
                    total_points += 1
    
        # Avoid division by zero
        if total_points == 0:
            return None, None
        
        self.centroid = (int(sum_x / total_points + self.bbox[0]), int(sum_y / total_points+self.bbox[1]))

    def __eq__(self, other):
        return self.centroid == other.centroid

    def __lt__(self, other):
        if abs(other.centroid[0] - self.centroid[0]) <= self.horizontal_threshold:
            if self.centroid[1] - other.centroid[1] < 0 :
                return True
            else :
                return False
        elif other.centroid[0] - self.centroid[0] > self.horizontal_threshold:
            return True
        elif self.centroid[0] - other.centroid[0] > self.horizontal_threshold:
            return False


class Window_partial_cells:
    def __init__(self, window_coordinates):
        self.type_4_pkts = []
        self.type_8_pkts = []
        self.window_coordinates = window_coordinates
   
    # type_4_pkts = []
    # type_8_pkts = []

In [5]:
class Cell_extractor(multiprocessing.Process):
    def __init__(self, cell_queue, partial_cell_queue, path, N_windows, window_event, extractor_finish_event):
        multiprocessing.Process.__init__(self)
        self.cell_queue = cell_queue  
        self.partial_cell_queue = partial_cell_queue  
        self.path = path
        self.N_windows = N_windows
        self.window_event = window_event
        self.extractor_finish_event = extractor_finish_event
    
    def run(self):
        self.name = multiprocessing.current_process().name

        image = read_input(self.path)
        im_row_length, im_col_length = image.shape
        num_window_row = 2
        num_window_col = 3
        win_row_length = int(im_row_length/num_window_row)
        win_col_length = int(im_col_length/num_window_col)
        
        total_windows = num_window_row*num_window_col

        i_windows = 0  # track how many windows have been processed
        
        for win_row_index in range(0,num_window_row):
            for win_col_index in range(0,num_window_col):
                # current_window = layout[win_row_index*win_row_length: (win_row_index+1)*win_row_length
                #                             ,win_col_index*win_col_length: (win_col_index+1)*win_col_length]
                start_row = win_row_index * win_row_length
                end_row = min((win_row_index + 1) * win_row_length, im_row_length)
                start_col = win_col_index * win_col_length
                end_col = min((win_col_index + 1) * win_col_length, im_col_length)
        
                current_window = image[start_row:end_row, start_col:end_col]
                
                # plt.imshow(current_window)
                # plt.show()
                img_bin = my_preprocessing(current_window)
                # plt.imshow(img_bin, cmap = 'gray')
                # plt.show()
                # perform prepcossing
                # layout, layout_denoised, layout_bin, th = pre_processing(layout_path)
                blobs_layout = connected_components(img_bin)
                window_coordinates = (win_row_index, win_col_index)
                window_partial_cells = Window_partial_cells( window_coordinates )

                for blob in blobs_layout:
                    blob_img = blob.image
                    # print(blob_img)
                    # plt.imshow(blob_img,cmap = 'gray')
                    # plt.show()
                    (min_row, min_col, max_row, max_col) = blob.bbox
                    # print(blob.bbox)
                    
                    pkt = shape_packet()
                    
                    pkt.bbox = (min_row+start_row, min_col+start_col, max_row+start_row, max_col+start_col) # we preserve coordinates 
                    pkt.blob_img = blob_img
                    # store centroid of shape coordinate in coordinate of original image
                    # print(start_row,start_col)
                    pkt.centroid = ( int( blob.centroid[0]+start_row), int(blob.centroid[1]+start_col)) # centroid from skii_image is wrt window image coordinates
                    # print(pkt.centroid)
                    # print(blob_img.shape)
                    
                    if ( max_col == win_col_length ) :
                        pkt.packet_type = 4
                        window_partial_cells.type_4_pkts.append(pkt)
                        # # print("found type",pkt.packet_type)
                    elif ( min_col == 0 ) :
                        pkt.packet_type = 8
                        window_partial_cells.type_8_pkts.append(pkt)
                        # print("found type",pkt.packet_type)
                    else:
                        # shape_pkts.append(pkt)
                        self.cell_queue.put(pkt)
                        # print("found type",pkt.packet_type)
                
                self.partial_cell_queue.put(window_partial_cells)
                
                i_windows = i_windows + 1

                if(i_windows == self.N_windows):
                    self.window_event.set()
                    i_windows = 0
        
        self.extractor_finish_event.set() # signal end of processing all the windows
        # print(f"{self.name} {self.pid}: all packets sent. Now exiting")
        

In [31]:
class Cell_collector(multiprocessing.Process):
    def __init__(self, cell_queue, sorted_cell_queue, N_cells, window_event, extractor_finish_event, merger_finish_event): 
        multiprocessing.Process.__init__(self)
        self.cell_queue = cell_queue  
        self.sorted_cell_queue = sorted_cell_queue
        self.N_cells = N_cells
        self.window_event = window_event
        self.extractor_finish_event = extractor_finish_event
        self.merger_finish_event = merger_finish_event
        self.sorted_cell_list = []
    
    def run(self):
        self.name = multiprocessing.current_process().name

        while True:
            if self.window_event.is_set():
                self.window_event.clear()                 
                if len(self.sorted_cell_list) == 0:
                    pass
                elif len(self.sorted_cell_list)> 0 and len(self.sorted_cell_list) < self.N_cells:
                    self.sorted_cell_queue.put(self.sorted_cell_list)
                    self.sorted_cell_list = []
                else:
                    self.sorted_cell_queue.put(self.sorted_cell_list[:self.N_cells])
                    del self.sorted_cell_list[:self.N_cells]
            
            # el
            if self.extractor_finish_event.is_set():
                print(len(self.sorted_cell_list))
                self.sorted_cell_queue.put(self.sorted_cell_list)
                self.sorted_cell_list = []
                self.extractor_finish_event.clear()
                
            #     # self.sorted_cell_queue.put(self.sorted_cell_list[:self.N_cells])
            #     # del self.sorted_cell_list[:self.N_cells]
                         
            
            # el
            if not self.cell_queue.empty():
                pkt = self.cell_queue.get()
                bisect.insort(self.sorted_cell_list , pkt)
                # print(len(self.sorted_cell_list))
                # if len(self.sorted_cell_list ) == 8:
                #     for xx in self.sorted_cell_list:
                #         plt.imshow( xx.blob_img )
                #         plt.show()  
            # print(f"{self.name}: {self.pid} Cell received with centroid= {pkt.centroid}")
            # plt.imshow( pkt.blob_img )
            # plt.show()

In [32]:
class Cell_merger(multiprocessing.Process):
    def __init__(self, partial_cell_queue, cell_queue, extractor_finish_event, merger_finish_event):
        multiprocessing.Process.__init__(self)
        self.partial_cell_queue = partial_cell_queue  
        self.cell_queue = cell_queue  
        self.extractor_finish_event = extractor_finish_event
        self.merger_finish_event = merger_finish_event
    
    def run(self):
        self.name = multiprocessing.current_process().name

        while True:

            if self.extractor_finish_event.is_set(): 
                if self.partial_cell_queue.empty(): # if queue is empty, signal finish event and terminate the process, else continue
                    self.merger_finish_event.set()
                    break
                else:
                    window_partial_cells = self.partial_cell_queue.get()
                    window_coordinates = window_partial_cells.window_coordinates
                    
                    if window_coordinates[1] == 0:    # Check if it is start of a row 
                        window_x = window_partial_cells
                    else:
                        window_x_1 = window_partial_cells
                        self.merge_cells(window_x, window_x_1)
                        window_x = window_x_1
            else:
                window_partial_cells = self.partial_cell_queue.get()
                window_coordinates = window_partial_cells.window_coordinates
                
                if window_coordinates[1] == 0:    # Check if it is start of a row 
                    window_x = window_partial_cells
                else:
                    window_x_1 = window_partial_cells
                    self.merge_cells(window_x, window_x_1)
                    window_x = window_x_1
            # print(f"{self.name}: {self.pid} Partial cell received with window coordinate= {window_partial_cells.window_coordinates}")
            # plt.imshow( pkt.blob_img )
            # plt.show()
    
    def merge_cells( self, p1, p2 ):
        
        for type_4_pkt in p1.type_4_pkts:
            type_4_img = type_4_pkt.blob_img
            # print(type_4_img)
            type_4_img_rows, type_4_img_cols = type_4_img.shape
            type_4_right_edge_min_row = float('inf')
            type_4_right_edge_max_row = float('-inf')
            # get the min & max row values of the pixels touching the right edge of the image
            for row_index in range(type_4_img_rows):
                if type_4_img[row_index][-1]:  # Check the last column of each row
                    type_4_right_edge_max_row = max(type_4_right_edge_max_row, row_index)
                    type_4_right_edge_min_row = min(type_4_right_edge_min_row, row_index)
            
            # print(" type4 org coordinate",type_4_right_edge_min_row,type_4_right_edge_max_row)
            # plt.imshow(type_4_img)
            # # print(type_4_img)
            # plt.show()
            # print(type_4_pkt.bbox)
                    
            # Shift to original coordinates
                    
            type_4_bbox_min_row, type_4_bbox_min_col, type_4_bbox_max_row, type_4_bbox_max_col = type_4_pkt.bbox
            type_4_right_edge_min_row = type_4_right_edge_min_row + type_4_bbox_min_row
            type_4_right_edge_max_row = type_4_right_edge_max_row + type_4_bbox_min_row
    
            # print(" type4 shifted coordinate",type_4_right_edge_min_row, type_4_right_edge_max_row)
            
            for type_8_pkt in p2.type_8_pkts[:]:
                type_8_img = type_8_pkt.blob_img
                type_8_img_rows, type_8_img_cols = type_8_img.shape
    
                # plt.imshow(type_8_img)
                # plt.show()
                
                type_8_left_edge_min_row = float('inf')
                type_8_left_edge_max_row = float('-inf')
                # get the min & max row values of the pixels touching the right edge of the image
                for row_index in range(type_8_img_rows):
                    if type_8_img[row_index][0]:  # Check the first column of each row
                        type_8_left_edge_max_row = max(type_8_left_edge_max_row, row_index)
                        type_8_left_edge_min_row = min(type_8_left_edge_min_row, row_index)
                
                # print("type 8 org",type_8_left_edge_min_row, type_8_left_edge_max_row)
                # Shift to original coordinates
                type_8_bbox_min_row, type_8_bbox_min_col , type_8_bbox_max_row, type_8_bbox_max_col = type_8_pkt.bbox
                type_8_left_edge_min_row = type_8_left_edge_min_row + type_8_bbox_min_row
                type_8_left_edge_max_row = type_8_left_edge_max_row + type_8_bbox_min_row
    
                # print("type8 shifted",type_8_left_edge_min_row, type_8_left_edge_max_row)
    
                #Check merging condition
                if (np.abs(type_4_right_edge_min_row - type_8_left_edge_min_row) <= 3) and (np.abs(type_4_right_edge_max_row - type_8_left_edge_max_row) <= 3):
                    
                    # print("match found")
    
                    # merge packets
                    merged_pkt =  shape_packet()
    
                    # new bounding box
                    merged_bbox_min_row = min( type_4_bbox_min_row, type_8_bbox_min_row)
                    merged_bbox_max_row = max( type_4_bbox_max_row, type_8_bbox_max_row)
                    merged_bbox_min_col = type_4_bbox_min_col
                    merged_bbox_max_col = type_8_bbox_max_col
    
                    merged_pkt.bbox = (merged_bbox_min_row,merged_bbox_min_col,merged_bbox_max_row,merged_bbox_max_col)
    
                    #new merged image
                    merged_image = np.zeros((merged_bbox_max_row-merged_bbox_min_row, merged_bbox_max_col-merged_bbox_min_col), dtype=bool)
    
                    row_diff = type_4_bbox_min_row - type_8_bbox_min_row
                    if ( row_diff > 0 ):
                        merged_image[ row_diff: row_diff+ type_4_img_rows, : type_4_img_cols] = type_4_img
                        merged_image[ : type_8_img_rows, type_4_img_cols: ] = type_8_img
                    else:
                        merged_image[ : type_4_img_rows, : type_4_img_cols ] = type_4_img
                        merged_image[ np.abs(row_diff): np.abs(row_diff)+ type_8_img_rows,  type_4_img_cols: ] = type_8_img
    
                    merged_pkt.blob_img = merged_image
                    merged_pkt.set_centroid()
                    
                    self.cell_queue.put(merged_pkt)

                    # shape_pkts.append(merged_pkt)
                    
                    # plt.imshow(merged_pkt.blob_img)
                    # plt.show()
                    
                    p2.type_8_pkts.remove(type_8_pkt)
                    
                    break

In [33]:
class Cell_validation(multiprocessing.Process):
    def __init__(self, sorted_cell_queue_layout, sorted_cell_queue_SEM):
        multiprocessing.Process.__init__(self)
        self.sorted_cell_queue_layout = sorted_cell_queue_layout  
        self.sorted_cell_queue_SEM = sorted_cell_queue_SEM  
    
    def run(self):
        self.name = multiprocessing.current_process().name

        while True:
            list = self.sorted_cell_queue_layout.get()
            print(f"{self.name}: {self.pid} list received with length= {len(list)}")
            # for pkt in list:
            #     plt.imshow(pkt.blob_img)
            #     plt.show()

In [34]:
if __name__ == '__main__':

    # Define parameters
    layout_name = "26.png"  # Replace with your image path 
    layout_path = os.path.join( os.getcwd(), 'images', layout_name)
    N_cells = 2
    N_windows = 1
    
    # Establish communication queues.
    cell_queue = multiprocessing.Queue()
    partial_cell_queue = multiprocessing.Queue()
    sorted_cell_queue_layout = multiprocessing.Queue()
    sorted_cell_queue_SEM = multiprocessing.Queue()
    window_event = multiprocessing.Event()
    extractor_finish_event = multiprocessing.Event()
    merger_finish_event = multiprocessing.Event()
    
    cell_extractor_layout = Cell_extractor(cell_queue, partial_cell_queue, layout_path, N_windows, window_event, extractor_finish_event)
    cell_collector_layout = Cell_collector(cell_queue, sorted_cell_queue_layout, N_cells, window_event, extractor_finish_event, merger_finish_event)
    cell_merger_layout = Cell_merger(partial_cell_queue, cell_queue, extractor_finish_event, merger_finish_event)

    cell_validation = Cell_validation(sorted_cell_queue_layout, sorted_cell_queue_SEM)
    
    cell_extractor_layout.start()
    cell_collector_layout.start()
    cell_merger_layout.start()
    cell_validation.start()
    
    cell_extractor_layout.join()
    cell_collector_layout.join()
    cell_merger_layout.join()

img shape:  (490, 1080)
 OT threshold:  127.0
 OT threshold:  127.0
 OT threshold:  127.0
 OT threshold:  126.0
 OT threshold:  125.0
 OT threshold:  127.0
7
Cell_validation-32: 149368 list received with length= 7


Process Cell_validation-32:
Process Cell_collector-30:
Process Cell_merger-31:


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/tmp/ipykernel_132258/237749113.py", line 41, in run
    if not self.cell_queue.empty():
  File "/tmp/ipykernel_132258/2502291265.py", line 11, in run
    list = self.sorted_cell_queue_layout.get()
  File "/tmp/ipykernel_132258/1533747021.py", line 11, in run
    window_partial_cells = self.partial_cell_queue.get()
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 129, in empty
    return not self._poll()
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 103, in get
    

In [123]:
my_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Cut the first 3 elements from the list
cut_elements = 3
new_list = my_list[cut_elements:]

print(my_list)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [124]:
del my_list
print(my_list)

NameError: name 'my_list' is not defined